In [17]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score

%matplotlib inline

# clean the data

In [3]:
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

df.drop([0, 1, 2], axis=0, inplace = True)
df.columns = df.iloc[0]
df.drop([3], axis=0, inplace = True)
df = df.reset_index(drop=True)
df.drop([348, 349, 350], axis=0, inplace = True)

df.rename(columns={ df.columns[0]: "city" }, inplace=True)
df.rename(columns={ df.columns[1]: "population" }, inplace=True)
df.rename(columns={ df.columns[2]: "violent_crime" }, inplace=True)
df.rename(columns={ df.columns[3]: "murder_nonnegligent_manslaughter" }, inplace=True)
df.rename(columns={ df.columns[4]: "rape_rev1" }, inplace=True)
df.rename(columns={ df.columns[5]: "rape_legacy" }, inplace=True)
df.rename(columns={ df.columns[6]: "robbery" }, inplace=True)
df.rename(columns={ df.columns[7]: "aggravated_assult" }, inplace=True)
df.rename(columns={ df.columns[8]: "property_crime" }, inplace=True)
df.rename(columns={ df.columns[9]: "burglary" }, inplace=True)
df.rename(columns={ df.columns[10]: "larceny_theft" }, inplace=True)
df.rename(columns={ df.columns[11]: "motor_vehicle_theft" }, inplace=True)
df.rename(columns={ df.columns[12]: "arson3" }, inplace=True)

df.drop(['rape_rev1'], axis=1, inplace = True)

df = df.fillna(0)

df['population'] = df['population'].astype('Int64')
df['violent_crime'] = df['violent_crime'].astype('Int64')
df['murder_nonnegligent_manslaughter'] = df['murder_nonnegligent_manslaughter'].astype('Int64')
df['rape_legacy'] = df['rape_legacy'].astype('Int64')
df['robbery'] = df['robbery'].astype('Int64')
df['aggravated_assult'] = df['aggravated_assult'].astype('Int64')
df['property_crime'] = df['property_crime'].astype('Int64')
df['burglary'] = df['burglary'].astype('Int64')
df['larceny_theft'] = df['larceny_theft'].astype('Int64')
df['motor_vehicle_theft'] = df['motor_vehicle_theft'].astype('Int64')
df['arson3'] = df['arson3'].astype('Int64')

# remove the outlier: New York
df.drop(216, inplace=True)

In [28]:
df.head()

3,city,population,violent_crime,murder_nonnegligent_manslaughter,rape_legacy,robbery,aggravated_assult,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson3,pop_sqrt,murder_bool,robbery_bool
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,43.139309,0,0
1,Addison Town and Village,2577,3,0,0,0,3,24,3,20,1,0,50.764161,0,0
2,Akron Village,2846,3,0,0,0,3,16,1,15,0,0,53.347915,0,0
3,Albany,97956,791,8,30,227,526,4090,705,3243,142,0,312.979233,1,1
4,Albion Village,6388,23,0,3,4,16,223,53,165,5,0,79.924965,0,1


In [29]:
df.describe()

3,population,violent_crime,murder_nonnegligent_manslaughter,rape_legacy,robbery,aggravated_assult,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson3,pop_sqrt,murder_bool,robbery_bool
count,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000
mean,15956.685879,51.213256,0.605187,2.677233,17.867435,30.063401,385.752161,72.172911,298.994236,14.585014,1.008646,104.719013,0.138329,0.599424
std,27080.218837,236.667435,3.707090,10.741020,94.972492,128.783376,1034.369072,264.941381,715.232296,67.682236,7.895813,70.746293,0.345743,0.490723
min,526.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.934690,0.000000,0.000000
25%,2997.000000,2.000000,0.000000,0.000000,0.000000,1.000000,40.000000,6.000000,31.000000,0.000000,0.000000,54.744753,0.000000,0.000000
50%,7187.000000,6.000000,0.000000,0.000000,1.000000,4.000000,112.000000,17.000000,94.000000,2.000000,0.000000,84.776176,0.000000,1.000000
75%,18160.500000,21.500000,0.000000,2.000000,5.000000,14.000000,340.500000,51.000000,284.500000,7.000000,0.000000,134.760876,0.000000,1.000000
max,258789.000000,3249.000000,47.000000,145.000000,1322.000000,1735.000000,12491.000000,3458.000000,8076.000000,957.000000,132.000000,508.713082,1.000000,1.000000


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 0 to 347
Data columns (total 15 columns):
city                                347 non-null object
population                          347 non-null int64
violent_crime                       347 non-null int64
murder_nonnegligent_manslaughter    347 non-null int64
rape_legacy                         347 non-null int64
robbery                             347 non-null int64
aggravated_assult                   347 non-null int64
property_crime                      347 non-null int64
burglary                            347 non-null int64
larceny_theft                       347 non-null int64
motor_vehicle_theft                 347 non-null int64
arson3                              347 non-null int64
pop_sqrt                            347 non-null float64
murder_bool                         347 non-null int64
robbery_bool                        347 non-null int64
dtypes: float64(1), int64(13), object(1)
memory usage: 53.4+ KB


# Model 1: 
**features that are not so great**

In [4]:
df['pop_sqrt'] = np.sqrt(df['population'])

df['murder_bool'] = 0
df['robbery_bool'] = 0

for index, row in df.iterrows():
    if row['murder_nonnegligent_manslaughter'] > 0:
        df.loc[index, 'murder_bool'] = 1

for index, row in df.iterrows():
    if row['robbery'] > 0:
        df.loc[index, 'robbery_bool'] = 1       

In [12]:
df.columns

Index(['city', 'population', 'violent_crime',
       'murder_nonnegligent_manslaughter', 'rape_legacy', 'robbery',
       'aggravated_assult', 'property_crime', 'burglary', 'larceny_theft',
       'motor_vehicle_theft', 'arson3', 'pop_sqrt', 'murder_bool',
       'robbery_bool'],
      dtype='object', name=3)

In [21]:
columns_1 = df[['population', 'pop_sqrt', 'murder_bool', 'robbery_bool']]
x_1 = columns_1
y_1 = df['property_crime']
x_1 = sm.add_constant(x_1)
results = sm.OLS(y_1,x_1).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         property_crime   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     405.5
Date:                Tue, 05 Mar 2019   Prob (F-statistic):          2.20e-128
Time:                        13:29:06   Log-Likelihood:                -2597.3
No. Observations:                 347   AIC:                             5205.
Df Residuals:                     342   BIC:                             5224.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          352.3650     65.600      5.371      0.000     223.334     481.396
population       0.0537      0.002     21.601      0.000       0.049       0.059
pop_sqrt        -8.9191      1.047     -8.516      0.000     -10.979      -6.859
murder_bool    139.4945     80.586      1.731      0.084     -19.013     298.002
robbery_bool   153.2653     59.050      2.596      0.010      37.118     269.413
==============================================================================
Omnibus:                      350.338   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            48843.185
Skew:                          -3.712   Prob(JB):                         0.00
Kurtosis:                      60.646   Cond. No.                     1.11e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Model 2: 
**using only the original variables as features**

In [22]:
df.columns

Index(['city', 'population', 'violent_crime',
       'murder_nonnegligent_manslaughter', 'rape_legacy', 'robbery',
       'aggravated_assult', 'property_crime', 'burglary', 'larceny_theft',
       'motor_vehicle_theft', 'arson3', 'pop_sqrt', 'murder_bool',
       'robbery_bool'],
      dtype='object', name=3)

In [24]:
columns_2 = df[['population', 'violent_crime', 'murder_nonnegligent_manslaughter', 'rape_legacy', 'robbery',
       'aggravated_assult', 'burglary', 'larceny_theft',
       'motor_vehicle_theft', 'arson3' ]]
x_2 = columns_2
y_2 = df['property_crime']
x_2 = sm.add_constant(x_2)
results = sm.OLS(y_2,x_2).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         property_crime   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.690e+30
Date:                Tue, 05 Mar 2019   Prob (F-statistic):               0.00
Time:                        13:31:50   Log-Likelihood:                 8546.8
No. Observations:                 347   AIC:                        -1.707e+04
Df Residuals:                     337   BIC:                        -1.704e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                             2.416e-13   3.44e-13      0.702      0.483   -4.35e-13    9.18e-13
population                       -2.862e-17   2.79e-17     -1.025      0.306   -8.35e-17    2.63e-17
violent_crime                    -7.105e-15   6.83e-14     -0.104      0.917   -1.41e-13    1.27e-13
murder_nonnegligent_manslaughter  5.684e-14   3.16e-13      0.180      0.857   -5.64e-13    6.78e-13
rape_legacy                       1.776e-15   1.48e-13      0.012      0.990    -2.9e-13    2.94e-13
robbery                          -1.776e-14   8.25e-14     -0.215      0.830    -1.8e-13    1.44e-13
aggravated_assult                -1.954e-14   6.71e-14     -0.291      0.771   -1.52e-13    1.13e-13
burglary                             1.0000   8.49e-15   1.18e+14      0.000       1.000       1.000
larceny_theft                        1.0000   1.83e-15   5.46e+14      0.000       1.000       1.000
motor_vehicle_theft                  1.0000   3.48e-14   2.88e+13      0.000       1.000       1.000
arson3                           -2.487e-14   5.49e-14     -0.453      0.651   -1.33e-13    8.32e-14
==============================================================================
Omnibus:                      563.124   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           134432.070
Skew:                           8.937   Prob(JB):                         0.00
Kurtosis:                      97.755   Cond. No.                     7.00e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.99e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

using all the original variables as features produces instead of the engineered categorical features increases the r-squared value from 0.826 to 1.

There are 3 features in this model with p-values = 0 
The other features have p-values much higher than .05

# Model 3: 
**using only high p-value features**

In [25]:
columns_3 = df[['population', 'violent_crime', 'murder_nonnegligent_manslaughter', 'rape_legacy', 'robbery',
       'aggravated_assult', 'arson3' ]]
x_3 = columns_3
y_3 = df['property_crime']
x_3 = sm.add_constant(x_3)
results = sm.OLS(y_3,x_3).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         property_crime   R-squared:                       0.954
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     1181.
Date:                Tue, 05 Mar 2019   Prob (F-statistic):          2.76e-224
Time:                        13:37:53   Log-Likelihood:                -2365.5
No. Observations:                 347   AIC:                             4745.
Df Residuals:                     340   BIC:                             4772.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               13.3960     15.286      0.876      0.381     -16.671      43.463
population                           0.0125      0.001     14.828      0.000       0.011       0.014
violent_crime                       12.4041      2.937      4.223      0.000       6.627      18.181
murder_nonnegligent_manslaughter     0.1805     14.293      0.013      0.990     -27.934      28.295
rape_legacy                         36.3107      6.227      5.831      0.000      24.063      48.558
robbery                            -12.5759      3.622     -3.472      0.001     -19.700      -5.452
aggravated_assult                  -11.5112      2.894     -3.977      0.000     -17.205      -5.818
arson3                              11.1496      2.274      4.904      0.000       6.677      15.622
==============================================================================
Omnibus:                      172.151   Durbin-Watson:                   2.068
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8482.018
Skew:                          -1.276   Prob(JB):                         0.00
Kurtosis:                      27.086   Cond. No.                     2.02e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.39e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In model_2, there were only 3 features with p-values less than 0.05

In model_3, those 3 features are removed.  Now, the same features which had high p-values in model_2, now have much lower p_values in model_3.

Even though model_3 is missing the best 3 features, it still results in a very good r-squared score of .954

Considering those 3 p-values where much better than any other features, it is suprising that their removal had such litte effect on the model.  

# Model 4:
**using only the lowest p-value features**

In [26]:
df.columns

Index(['city', 'population', 'violent_crime',
       'murder_nonnegligent_manslaughter', 'rape_legacy', 'robbery',
       'aggravated_assult', 'property_crime', 'burglary', 'larceny_theft',
       'motor_vehicle_theft', 'arson3', 'pop_sqrt', 'murder_bool',
       'robbery_bool'],
      dtype='object', name=3)

In [27]:
columns_4 = df[['burglary', 'larceny_theft','motor_vehicle_theft' ]]
x_4 = columns_4
y_4 = df['property_crime']
x_4 = sm.add_constant(x_4)
results = sm.OLS(y_4,x_4).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         property_crime   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.229e+32
Date:                Tue, 05 Mar 2019   Prob (F-statistic):               0.00
Time:                        13:49:53   Log-Likelihood:                 9096.9
No. Observations:                 347   AIC:                        -1.819e+04
Df Residuals:                     343   BIC:                        -1.817e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -3.553e-14   6.27e-14     -0.567      0.571   -1.59e-13    8.77e-14
burglary                1.0000   1.22e-15   8.21e+14      0.000       1.000       1.000
larceny_theft           1.0000   2.43e-16   4.11e+15      0.000       1.000       1.000
motor_vehicle_theft     1.0000   3.73e-15   2.68e+14      0.000       1.000       1.000
==============================================================================
Omnibus:                      585.965   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           168808.328
Skew:                           9.602   Prob(JB):                         0.00
Kurtosis:                     109.333   Cond. No.                         955.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""